In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [13]:
!pip install -U scikit-learn

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
%cd /content/drive/MyDrive/pipe/lab4sem

/content/drive/MyDrive/pipe/lab4sem


In [22]:
data = pd.read_csv('Laptop_price.csv')

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             1000 non-null   object 
 1   Processor_Speed   1000 non-null   float64
 2   RAM_Size          1000 non-null   int64  
 3   Storage_Capacity  1000 non-null   int64  
 4   Screen_Size       1000 non-null   float64
 5   Weight            1000 non-null   float64
 6   Price             1000 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 54.8+ KB


In [24]:
y = data[['Price']]

In [25]:
X = data.drop('Price', axis=1)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

In [27]:
num_columns = list(X_train.select_dtypes(exclude=['object']).columns)
num_columns

['Processor_Speed', 'RAM_Size', 'Storage_Capacity', 'Screen_Size', 'Weight']

In [28]:
cat_columns = list(X_train.select_dtypes(include=['object']).columns)
cat_columns

['Brand']

In [29]:
numerical = Pipeline(steps=[
    ("simtle_imputer", SimpleImputer(strategy='constant')),
    ("power_transform", PowerTransformer()),
    ('Scaler',StandardScaler())
])

In [30]:
categorical = Pipeline(steps=[
    ("simple_limiter", SimpleImputer(strategy='constant')),
    ("OneHotEncoder", OneHotEncoder(handle_unknown='ignore',
                                    sparse_output=False))
])

In [31]:
ct = ColumnTransformer([
    ("numerical", numerical, num_columns ),
     ("categorical", categorical, cat_columns )
])

In [32]:
pipe = Pipeline(steps=[
    ("ct", ct),
    ("XGBRegressor", XGBRegressor())
])

In [34]:
param_grid={
    'XGBRegressor__n_estimators' : [50],
    'XGBRegressor__max_depth' : [3],
    'XGBRegressor__gamma' : [6],
    'XGBRegressor__max_leaves' : [0],
    'XGBRegressor__learning_rate' : [1],
    'ct__numerical' : [StandardScaler(), RobustScaler()]
}

In [35]:
clf = GridSearchCV(pipe, param_grid, verbose=3)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.6s
[CV 2/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.2s
[CV 3/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.0s
[CV 4/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.0s
[CV 5/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRe

GridSearchCV(estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('simtle_imputer',
                                                                                          SimpleImputer(strategy='constant')),
                                                                                         ('power_transform',
                                                                                          PowerTransformer()),
                                                                                         ('Scaler',
                                                                                          StandardScaler())]),
                                                                         ['Processor_Speed',
                                                                          'RAM_Size',
                                                                          'Storage_Capacity',
                                                                          'Screen_Size',
                                                                          'Weight']),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('simple_limiter...
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             param_grid={'XGBRegressor__gamma': [6],
                         'XGBRegressor__learning_rate': [1],
                         'XGBRegressor__max_depth': [3],
                         'XGBRegressor__max_leaves': [0],
                         'XGBRegressor__n_estimators': [50],
                         'ct__numerical': [StandardScaler(), RobustScaler()]},
             verbose=3)

In [ ]:
#model=clf.best_estimator_

In [ ]:
#clf.best_params_

{'XGBRegressor__gamma': 6,
 'XGBRegressor__learning_rate': 1,
 'XGBRegressor__max_depth': 3,
 'XGBRegressor__max_leaves': 0,
 'XGBRegressor__n_estimators': 50,
 'ct__numerical': StandardScaler()}

In [36]:
pip install mlflow

In [37]:
pip install optuna


In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.model_selection import GridSearchCV

In [51]:
%cd /content/drive/MyDrive/pipe/lab4sem

/content/drive/MyDrive/pipe/lab4sem


In [52]:
data = pd.read_csv('bank.csv')

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                        Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                        --------------  ----- 
 0   age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"  41188 non-null  object
dtypes: object(1)
memory usage: 321.9+ KB


In [54]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [57]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_int('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    model = XGBClassifier(**params)

    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_score = study.best_value

[I 2024-05-16 13:32:38,299] A new study created in memory with name: no-name-81b72811-87d3-4b20-a0fe-298445f22682
<ipython-input-57-7f6717c35803>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
<ipython-input-57-7f6717c35803>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
<ipython-input-57-7f6717c35803>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
[W 2024-05-16 

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1471, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  4

In [58]:
import mlflow

with mlflow.start_run():
    bank= study.best_params
    mlflow.log_params(bank)
    mlflow.log_metric('accuracy', best_score)

ValueError: No trials are completed yet.

In [62]:
#lab 4
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9f3152f3860098c5f5dd22ea4728ffb51366f19e49a33630a3352aa845adcb72
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [63]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

In [64]:
APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

In [65]:
spark = SparkSession.builder.appName(APP_NAME) \
        .config('spark.ui.showConsoleProgress', 'false') \
        .getOrCreate()

In [66]:
%cd /content/drive/MyDrive/pipe/lab4sem

/content/drive/MyDrive/pipe/lab4sem


In [67]:
taxi = spark.read.load('taxi.csv',
                       format='csv', header='true', inferSchema='true')

In [68]:
taxi.registerTempTable("taxi")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [69]:
print("Количество объектов в датафрейме:", taxi.count())

Количество объектов в датафрейме: 128974


In [70]:
taxi.show(n=5)

+----------+----+------+-------+
|      date|hour|minute|pickups|
+----------+----+------+-------+
|2009-01-01|   0|     0|   24.0|
|2009-01-01|   0|    30|   35.0|
|2009-01-01|   1|     0|   25.0|
|2009-01-01|   1|    30|   25.0|
|2009-01-01|   2|     0|   16.0|
+----------+----+------+-------+
only showing top 5 rows



In [71]:
taxi['date', 'hour', 'minute'].show(5)

+----------+----+------+
|      date|hour|minute|
+----------+----+------+
|2009-01-01|   0|     0|
|2009-01-01|   0|    30|
|2009-01-01|   1|     0|
|2009-01-01|   1|    30|
|2009-01-01|   2|     0|
+----------+----+------+
only showing top 5 rows



In [72]:
taxi.describe().show()

+-------+------------------+------------------+------------------+
|summary|              hour|            minute|           pickups|
+-------+------------------+------------------+------------------+
|  count|            128974|            128974|            128969|
|   mean|11.566509529052366|15.004419495402175|29.009451883786028|
| stddev| 6.908556452594711|15.000057500526209|  22.4493784836831|
|    min|                 0|                 0|               1.0|
|    max|                23|                30|             310.0|
+-------+------------------+------------------+------------------+



In [73]:
print(taxi.summary().show())


+-------+------------------+------------------+------------------+
|summary|              hour|            minute|           pickups|
+-------+------------------+------------------+------------------+
|  count|            128974|            128974|            128969|
|   mean|11.566509529052366|15.004419495402175|29.009451883786028|
| stddev| 6.908556452594711|15.000057500526209|  22.4493784836831|
|    min|                 0|                 0|               1.0|
|    25%|                 6|                 0|              11.0|
|    50%|                12|                30|              27.0|
|    75%|                18|                30|              40.0|
|    max|                23|                30|             310.0|
+-------+------------------+------------------+------------------+

None


In [74]:
taxi.fillna(0).describe().show()

+-------+------------------+------------------+------------------+
|summary|              hour|            minute|           pickups|
+-------+------------------+------------------+------------------+
|  count|            128974|            128974|            128974|
|   mean|11.566509529052366|15.004419495402175| 29.00832725975778|
| stddev| 6.908556452594711|15.000057500526209|22.449669931429067|
|    min|                 0|                 0|               0.0|
|    max|                23|                30|             310.0|
+-------+------------------+------------------+------------------+



In [75]:
print('Количество объектов без пропусков до удаления пропусков')
print('\nДо удаления пропусков   :', taxi.dropna(how='any', subset='pickups').count())

Количество объектов без пропусков до удаления пропусков

До удаления пропусков   : 128969


In [76]:
taxi = taxi.fillna(0)

In [77]:
print('После удаления пропусков:', taxi.dropna(how='any', subset='pickups').count())

После удаления пропусков: 128974


In [78]:
result = spark.sql("""
SELECT   *
FROM     taxi
ORDER BY pickups DESC
--LIMIT    5
;
""")

print(result.show(5))

+----------+----+------+-------+
|      date|hour|minute|pickups|
+----------+----+------+-------+
|2015-11-01|   1|    30|  310.0|
|2010-09-23|  22|    30|  288.0|
|2012-03-07|  21|     0|  268.0|
|2011-03-02|  20|    30|  264.0|
|2011-03-02|  18|    30|  263.0|
+----------+----+------+-------+
only showing top 5 rows

None


In [79]:
result = spark.sql("""
SELECT   COUNT(DISTINCT date)
FROM     taxi
WHERE    pickups > 200
;
""")

print(result.show())

+--------------------+
|count(DISTINCT date)|
+--------------------+
|                  21|
+--------------------+

None


In [80]:
print(spark.sql("""
SELECT EXTRACT(MONTH FROM date),
       AVG(pickups)
FROM   taxi
GROUP BY EXTRACT(MONTH FROM date)
ORDER BY AVG(pickups) DESC;
""").show())


+------------------------+------------------+
|extract(MONTH FROM date)|      avg(pickups)|
+------------------------+------------------+
|                       3| 34.61413319776309|
|                      10|31.492839171666343|
|                       2|29.856671982987773|
|                       5| 29.81593638978176|
|                       4|29.313725490196077|
|                       9|29.158446485623003|
|                      11|28.860367558929283|
|                       1|28.559511612021858|
|                       6| 27.03835736129314|
|                       7| 26.45983005021244|
|                      12| 26.45916884626562|
|                       8| 25.88592750533049|
+------------------------+------------------+

None


In [81]:
print(spark.sql("""
SELECT   hour,
         AVG(pickups)
FROM     taxi
GROUP BY hour
ORDER BY AVG(pickups) DESC
""").show(10))

+----+------------------+
|hour|      avg(pickups)|
+----+------------------+
|   8| 48.98208348725527|
|   9| 45.74220335855324|
|  18|45.131967515688444|
|  19| 40.18456995201181|
|  17| 37.68493909191584|
|  12| 36.91678966789668|
|  10|36.391031555637575|
|  14|35.965867158671585|
|   7| 35.94376618571957|
|  13| 35.34939091915836|
+----+------------------+
only showing top 10 rows

None
